In [1]:
import pandas as pd

In [2]:
def load_data_from_api(url, ext, months):
    """
    Template for loading data from API
    """
    df = pd.DataFrame()

    for i in months:
        # load file with pd.concat
        

        #  df = pd.concat([df, pq.read_table(f'{url}{i}{ext}').to_pandas()],
        #         ignore_index=True)
        df = pd.concat([df, pd.read_parquet(f'{url}{i}{ext}',
        engine= 'pyarrow',
        )], ignore_index=True)
    
    taxi_dtypes = {
                    'VendorID': pd.Int64Dtype(),
                    'tpep_pickup_datetime': 'datetime64[s]',
                    'tpep_dropoff_datetime': 'datetime64[s]',
                    'passenger_count': pd.Int64Dtype(),
                    'trip_distance': float,
                    'RatecodeID':pd.Int64Dtype(),
                    'store_and_fwd_flag':str,
                    'PULocationID':pd.Int64Dtype(),
                    'DOLocationID':pd.Int64Dtype(),
                    'payment_type': pd.Int64Dtype(),
                    'fare_amount': float,
                    'extra':float,
                    'mta_tax':float,
                    'tip_amount':float,
                    'tolls_amount':float,
                    'improvement_surcharge':float,
                    'total_amount':float,
                    'congestion_surcharge':float
                }
    
    df = df.astype(dtype=taxi_dtypes)
    
    return df

## Q1. Downloading the data

In [3]:
# Download the data of January 2023
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-'
ext = '.parquet'
months = ['01'] 
df_train = load_data_from_api(url, ext, months)

# Download the data of February 2023
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-'
ext = '.parquet'
months = ['02'] #['01','02','03','04','05','06','07','08','09','10','11','12']
df_val = load_data_from_api(url, ext, months)

In [4]:
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1,0.97,1,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1,1.10,1,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1,2.51,1,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0,1.90,1,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1,1.43,1,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
len(df_train.columns)

19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [6]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
    # Convert from seconds to minutes
df_train.duration = df_train.duration.apply(lambda x: x.total_seconds() / 60)

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
    # Convert from seconds to minutes
df_val.duration = df_val.duration.apply(lambda x: x.total_seconds() / 60)

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?


In [7]:
df_train['duration'].describe(percentiles=[.90, .95, .98])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
50%      1.151667e+01
90%      2.794167e+01
95%      3.646667e+01
98%      4.873333e+01
max      1.002918e+04
Name: duration, dtype: float64

In [8]:
old_length = df_train.shape[0]
old_length

3066766

In [9]:
df_train = df_train[(df_train.duration >=1) & (df_train.duration <= 60)]
df_val = df_val[(df_val.duration >=1) & (df_val.duration <= 60)]

In [10]:
new_length = df_train.shape[0]
new_length

3009173

Ratio retained

In [12]:
new_length/old_length

0.9812202822125979

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?


In [21]:
# Turn 'PULocationID' and  'DOLocationID' to string 

categorical = ['PULocationID', 'DOLocationID']

df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)


In [14]:
from sklearn.feature_extraction import DictVectorizer

In [22]:
dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
X_train.shape

(3009173, 515)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [28]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [29]:
lr = LinearRegression()
lr.fit(X_train, y_train)
ytrain_pred = lr.predict(X_train)
mean_squared_error(y_train, ytrain_pred, squared=False)


7.649261929865273

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [30]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.81181531443233